In [1]:
import sys
import xarray as xr
import numpy as np
import pandas as pd

# import wrf


# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from utils import roundPartial

In [4]:
fname = '/expanse/lustre/scratch/dnash/temp_project/mclimate/freezing_level/20010528_freezing_level_F99_F120.nc'
# fname = '/home/dnash/DATA/preprocessed/GEFSv12_reforecast/freezing_level/20010528_freezing_level_F99_F120.nc'
ds = xr.open_dataset(fname)
ds

<xarray.Dataset> Size: 43MB
Dimensions:         (number: 5, step: 8, lat: 281, lon: 479)
Coordinates:
  * number          (number) int64 40B 0 1 2 3 4
  * step            (step) timedelta64[ns] 64B 4 days 03:00:00 ... 5 days 00:...
  * lat             (lat) float64 2kB 70.0 69.75 69.5 69.25 ... 0.5 0.25 0.0
  * lon             (lon) float64 4kB -179.5 -179.2 -179.0 ... -60.25 -60.0
    valid_time      (step) datetime64[ns] 64B ...
Data variables:
    freezing_level  (number, step, lat, lon) float64 43MB ...

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
## create bounding boxes for regions
d = {'baja': {'ext': [-115., -110., 27., 32.],
              'start_pt': [35.0, -120.],
              'end_pt': [30.0, -105.]},
     
     'san_juan': {'ext': [-110., -105., 35., 40.],
              'start_pt': [37.0, -112.],
              'end_pt': [37.0, -102.]}
    }
                  
## create a dataset with lats and lons from ext
ext = d['baja']['ext']
lats = np.arange(ext[2], ext[3]+0.25, 0.25)
lons = np.arange(ext[0], ext[1]+0.25, 0.25)

print(lats)
print(lons)

[27.   27.25 27.5  27.75 28.   28.25 28.5  28.75 29.   29.25 29.5  29.75
 30.   30.25 30.5  30.75 31.   31.25 31.5  31.75 32.  ]
[-115.   -114.75 -114.5  -114.25 -114.   -113.75 -113.5  -113.25 -113.
 -112.75 -112.5  -112.25 -112.   -111.75 -111.5  -111.25 -111.   -110.75
 -110.5  -110.25 -110.  ]


In [4]:
# ## create a simple map to show the areas

# ax_ctt.plot([start_point.lon, end_point.lon],
#             [start_point.lat, end_point.lat], color="yellow", marker="o",
#             transform=crs.PlateCarree(), zorder=3)

In [6]:
## load PRISM watershed precip dataset
fname = path_to_data + 'preprocessed/PRISM/PRISM_HUC8_CO_sp.nc'
PRISM = xr.open_dataset(fname)
HUC8_lst = PRISM.HUC8.values ## get list of HUC8 IDs

ds_lst = []
## load final trajectory dataset
for i, HUC8_ID in enumerate(HUC8_lst):
    fname = path_to_data + 'preprocessed/ERA5_trajectories/combined_extreme_AR/PRISM_HUC8_{0}.nc'.format(HUC8_ID)
    traj = xr.open_dataset(fname)
    ds_lst.append(traj)

## concat ds_lst along HUC8 index
ds = xr.concat(ds_lst, pd.Index(HUC8_lst, name="HUC8"))
ds

<xarray.Dataset> Size: 1GB
Dimensions:             (index: 72, start_date: 1641, HUC8: 92)
Coordinates:
  * index               (index) int64 576B 0 1 2 3 4 5 6 ... 66 67 68 69 70 71
  * start_date          (start_date) datetime64[ns] 13kB 2000-01-11 ... 2023-...
    time                (HUC8, start_date, index) datetime64[ns] 87MB NaT ......
    lon                 (HUC8, start_date, index) float64 87MB nan nan ... nan
    lat                 (HUC8, start_date, index) float64 87MB nan nan ... nan
  * HUC8                (HUC8) object 736B '14080107' '14080105' ... '10250013'
Data variables: (12/18)
    level               (HUC8, start_date, index) float64 87MB nan nan ... nan
    q                   (HUC8, start_date, index) float64 87MB nan nan ... nan
    u                   (HUC8, start_date, index) float64 87MB nan nan ... nan
    v                   (HUC8, start_date, index) float64 87MB nan nan ... nan
    w                   (HUC8, start_date, index) float64 87MB nan nan ... nan
    IVT                 (HUC8, start_date, index) float64 87MB nan nan ... nan
    ...                  ...
    coastal_IVT         (HUC8, start_date) float64 1MB nan nan nan ... nan nan
    ar_scale_strict     (HUC8, start_date) float64 1MB nan nan nan ... nan nan
    ar_strict           (HUC8, start_date) float64 1MB nan nan nan ... nan nan
    tARget_strict       (HUC8, start_date) float64 1MB nan nan nan ... nan nan
    coastal_IVT_strict  (HUC8, start_date) float64 1MB nan nan nan ... nan nan
    time_match          (HUC8, start_date) object 1MB nan nan nan ... nan nan

In [9]:
def flatten(xss):
    return [x for xs in xss for x in xs]
    
def find_time_bbox(ERA5, lats, lons):
    ## a function that gets the start dates and HUC8 ID 
    ## for the times when a trajectory is within the bbox
    
    ## create a dataset of the trajectory points that match ERA5 spacing
    t = xr.DataArray(ERA5.time.values, dims=['location'], name='time')
    
    # create a list of lat/lons that match ERA5 spacing
    x = xr.DataArray(roundPartial(ERA5.lon.values, 0.25), dims=['location'])
    y = xr.DataArray(roundPartial(ERA5.lat.values, 0.25), dims=['location'])
    
    x = xr.DataArray(ERA5.lon.values, dims=("location"), coords={"lon": x}, name='traj_lons')
    y = xr.DataArray(ERA5.lat.values, dims=("location"), coords={"lat": y}, name='traj_lats')
    
    # create a new dataset that has the trajectory lat and lons and the closest ERA5 lat/lons as coords
    z = xr.merge([x, y, t])
    
    ## Now loop through the lat/lon pairs and see where they match
    idx_lst = []
    for i, (x, y) in enumerate(zip(z.lon.values, z.lat.values)):
        for j, lon in enumerate(lons):
            for k, lat in enumerate(lats):
            
                ## test if lat/lon pair matches
                result_variable = (lon == x) & (lat == y)
        
                if (result_variable == True):
                    idx = (i, j, k) # (index of z, index of txtpts)
                    idx_lst.append(idx)
    ts_lst = []
    if len(idx_lst) > 0:
        for m, idx in enumerate(idx_lst):
            ## this is the time of the trajectory when it crosses west coast
            time_match = z.sel(location=idx_lst[m][0]).time.values
            ts = pd.to_datetime(str(time_match)).strftime('%Y-%m-%d %H')
            ts_lst.append(ts)

    return ts_lst

In [14]:
df_lst = []
## loop through HUC8s and start_dates
for i, HUC8 in enumerate(HUC8_lst):
    # subset to the current HUC8
    ## keep only trajectories associated with ARs
    tmp = ds.sel(HUC8=HUC8)
    tmp = tmp.where(tmp.ar_scale > 0, drop=True)


    t_lst = []
    ## enumerate through start_dates of current subbasin
    for i, st_date in enumerate(tmp.start_date.values):
        ERA5 = tmp.sel(start_date=st_date)
        test = find_time_bbox(ERA5, lats, lons)
        t_lst.append(test)
    
    t_lst = flatten(t_lst)
    df = pd.DataFrame(data={'date': t_lst}) # put dates into a df
    # t_lst = pd.to_datetime(t_lst, format='%Y-%m-%d %H')
    df_lst.append(df)

In [24]:
## now we have a list of list of dates when AR trajectories were within bbox
## concat df_lst
df = pd.concat(df_lst)
## swap to datetime
df['datetime'] = pd.to_datetime(df['date'])

## remove duplicates
df = df.drop_duplicates(subset=['datetime'])
df = df.drop(['date'], axis=1)

## save as CSV dates_region-name.csv
fname_out = '../out/bbox_dates_baja.csv'
df.to_csv(fname_out)

In [ ]:
# # Set the start point and end point for the cross section
# start_point = wrf.CoordPair(lat=26.76, lon=-80.0)
# end_point = wrf.CoordPair(lat=26.76, lon=-77.8)

# # interpolate the coord_pairs on ERA5
# coord_pairs = ter.coords["xy_loc"].values
# yvals = [pair.lat for pair in coord_pairs]
# xvals = [pair.lon for pair in coord_pairs]
# x = xr.DataArray(xvals, dims="z")
# y = xr.DataArray(yvals, dims="z")
# cross.append(data.interp(lon=x, lat=y))